In [41]:
import spacy
from spacy.tokens import Doc, Token
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib
import time
from typing import Optional

import ogree_wiki as wiki
importlib.reload(wiki)

<module 'ogree_wiki' from 'c:\\Users\\thoxy\\OneDrive\\Documents\\IMT-PJENT\\OGrEE_NLP\\ogree_wiki.py'>

In [7]:
ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move","set","rename","rotate"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_THRESHOLD = 0.5

PARAMETERS_DICT = {
            "name" : ["name","called"],
            "position" : ["position","at","located"],
            "rotation" : ["rotation","turned","degree"],
            "size" : ["size","dimensions"],
            "template" : ["template"],
            "axisOrientation" : ["orientation"], #CAN BE IMPROVED !!!!!!!!!
            "slot" : ["slot"],
            "color" : ["color"], #Est-ce qu'on rajoute des couleurs genre red, green, grey, ...?
            "unit" :  ["unit"],    # C'est quoi le mieux ? ["t","m","f",'metre', "tilde", "feet"],
            "side" : ["side"],
            "temperature" : ["temp","temperature", "°C", "K","F"],
            "reserved" : ["reserved"],
            "technical" : ["technical"]
            }

In [49]:
liste = [1,2,3]
liste.extend([])
liste


[1, 2, 3]

In [96]:
def findIndexMainSubject(processed_entry : Doc, dictioIndexKeyWords : dict, indexAction : int, indexMainEntity : int = None) -> int :

    def searchSubjectRecursive(processed_entry : Doc, currentIndex : int, testConformity, level : int = 0) -> (list|None):
        if level == 5 :
            return None
        if testConformity(processed_entry[currentIndex]) :
            return [(currentIndex, level)]
        else:
            childList = []
            for child in processed_entry[currentIndex].children :
                childResult = searchSubjectRecursive(processed_entry, child.i, testConformity, level+1)
                if childResult != None :
                    childList.extend(childResult)
            if bool(childList) == True : # if the list is not empty    
                minValue = min(childList, key=lambda x: x[1])[1]
                return [x for x in childList if x[1] == minValue] 
            return childList
        
    def testConformityMain(token : Token) -> bool :
        if token.i in dictioIndexKeyWords.keys() and token.pos_ != "VERB":
            return True
        return False
    
    def testConformityAlteration(token : Token) -> bool :
        if ((token.i in dictioIndexKeyWords.keys() or token.pos_ == "NOUN") 
            and token.pos_ != "VERB"
            and not token.is_upper) :
            return True
        return False
    
    actionType = dictioIndexKeyWords[indexAction]
    testConformity = testConformityAlteration if actionType == "ALTERATION" else testConformityMain

    result = searchSubjectRecursive(processed_entry, indexAction, testConformity)
    result = [x[0] for x in result]
    resultLength = len(result)

    if resultLength == 0 :
        raise Exception("Main request not identified")
    elif resultLength == 1 :
        return result[0]
    else : # if there are at least two tokens at the same length from the action

        if actionType == "ALTERATION" : # if alteration, prioritize known parameters
            resultOnlyParameters = [index for index in result if dictioIndexKeyWords[index] in PARAMETERS_DICT.keys()]
            if bool(resultOnlyParameters) :
                return resultOnlyParameters[0]
            else :
                return result[0]
            
        else : # prioritize entities
            resultOnlyEntity = [index for index in result if dictioIndexKeyWords[index] == "entity"]
            if not bool(resultOnlyEntity) :
                return result[0]
            elif len(resultOnlyEntity) == resultLength and indexMainEntity in result :
                return indexMainEntity
            else :
                return resultOnlyEntity[0]


In [56]:
# test if the relation is coherent, according to the relation
def testRelation(index1 : int, index2 : int, relationType : str, hierarchyPosition : dict) -> bool :
    if relationType == "hierarchy" and hierarchyPosition[index2] < hierarchyPosition[index1]:
        return True
    elif relationType == "location" and hierarchyPosition[index2] == hierarchyPosition[index1]:
        return True
    return False

def findIndexMainEntity(processed_entry : Doc, dictEntities : dict, indexAction : int) -> int :
    counter = 0
    currentIndexes = {index:index for index in dictEntities.keys()}
    currentWords = {index:processed_entry[index] for index in currentIndexes.keys()}

    while (not indexAction in currentIndexes.values()) and counter < 3 :
        currentWords = {originIndex : processed_entry[currentIndex].head for originIndex,currentIndex in currentIndexes.items()}
        currentIndexes = {originIndex : currentWords[originIndex].i for originIndex,_ in currentIndexes.items()}

        if list(currentIndexes.values()).count(indexAction) == 1 :
            return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]
        counter += 1

    if counter == 3 :
        raise Exception("Main entity not found")
    
    if list(currentIndexes.values()).count(indexAction) != 1 :
        listIndexesRemaining = [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction and originIndex > indexAction]
        return listIndexesRemaining[0]
    else :
        return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]


def findRelations(processed_entry : Doc, dictEntities : dict, indexAction : int) -> dict :

    RELATIONS = {
        "hierarchy" : ["in", "inside"],
        "location" : ["next"]
    }
    
    dictRelations = {index : None for index in dictEntities.keys()} # empty dict that will be filled

    # go through the ancestors and check if there's a synonym of the relation key words
    for index in dictEntities.keys() :
        for ancestor in processed_entry[index].ancestors :
            for relation in RELATIONS.keys() :
                if max([ancestor.similarity(nlp(word)[0]) > SIMILARITY_THRESHOLD for word in RELATIONS[relation]]) :
                    dictRelations[index] = relation
                    break

    global INDEX_MAIN_ENTITY

    # if zero or more than 1 entities don't have a relation
    withoutRelationCounter = list(dictRelations.values()).count(None)
    if  withoutRelationCounter != 1 :
        dictWithoutRelations = {index : relation for (index,relation) in dictRelations.items() if relation == None}
        if withoutRelationCounter == 0 :
            dictWithoutRelations = dictEntities
        INDEX_MAIN_ENTITY = findIndexMainEntity(processed_entry, dictWithoutRelations, indexAction)

    # if only one entity is not attached to a relation keyword, it's the main one
    INDEX_MAIN_ENTITY = [index for index,relation in dictRelations.items() if relation == None][0]

    # the hierarchy position : 0 is site, 3 is rack... etc
    hierarchyPosition = {index : list(wiki.ENTITIES.keys()).index(entity) for index, entity in dictEntities.items()}
    finalRelations = dict()

    # for all entities except the main one, we assign the related entity according to the relation
    for index, relation in dictRelations.items() : 
        if index == INDEX_MAIN_ENTITY:
            continue
        for token in processed_entry[index].ancestors :
            if token.i in dictEntities.keys(): 
                if testRelation(token.i, index, relation, hierarchyPosition):
                    finalRelations[index] = (token.i, relation)
                else:
                    finalRelations[index] = (token.i, "ERROR")
                break
            elif token.i == indexAction:
                if testRelation(INDEX_MAIN_ENTITY, index, relation, hierarchyPosition):
                    finalRelations[index] = (INDEX_MAIN_ENTITY, relation)
                else:
                    finalRelations[index] = (INDEX_MAIN_ENTITY, "ERROR")
                break
        if not index in finalRelations :
            finalRelations[index] = (INDEX_MAIN_ENTITY, relation)

    return finalRelations

In [53]:
def name(processed_entry : Doc, dictioEntities : dict, listNameSynonyms : list, indexesMain : dict) -> int :

    def findClose(processed_entry : Doc, index : int) -> (int|None) :
        if index +1 <= len(processed_entry)-1 :
            if processed_entry[index+1].is_upper or processed_entry[index+1].pos_ == "PROPN" :
                return index+1
        if 0 <= index -1 :
            if processed_entry[index-1].is_upper or processed_entry[index-1].pos_ == "PROPN" :
                return index-1
        return None

    def findAttachedEntity(processed_entry : Doc, index : int) -> (int|None) : 
        counter = 0
        for token in processed_entry[index].ancestors :
            if counter == 3 :
                break
            if token.i in dictioEntities.keys() :
                return token.i
            if token.i == indexesMain["action"] :
                return indexesMain["entity"]
            counter += 1
        return None

    EXPLICIT =  PARAMETERS_DICT["name"]
    IMPLICIT = ["current","main"]

    dictioEntityNames = {} # dict with entityIndex : NameOfTheEntityIndex

    # begin with the synonyms of "called"
    for nameSynonymIndex in listNameSynonyms :
        currentToken = processed_entry[nameSynonymIndex]
        if len(list(currentToken.ancestors)) + len(list(currentToken.children)) == 0 :
            continue
        else :
            attachedEntityIndex = findAttachedEntity(processed_entry, nameSynonymIndex)
            attachedValueIndex = list(currentToken.children)[0]
        
        if attachedEntityIndex != None :
            dictioEntityNames[attachedEntityIndex] = attachedValueIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        # if the name if right beside the entity
        for entityIndex,_ in dictioEntities.items() :
            if entityIndex in dictioEntityNames.keys() :
                continue
            attachedValueIndex = findClose(processed_entry, entityIndex)
            if attachedValueIndex != None :
                dictioEntityNames[entityIndex] = attachedValueIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        for index,token in enumerate(processed_entry) : # look directly for a name

            if processed_entry[index].is_upper or processed_entry[index].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(processed_entry,index)
                if not (indexAttachedEntity in dictioEntityNames.keys() or indexAttachedEntity == None) :
                    dictioEntityNames[indexAttachedEntity] = index

    if len(dictioEntityNames) < len(dictioEntities) : # if still not all names found
        pass
    # TODO : get current names

    # TODO : check for implicit words now
    
    return dictioEntityNames

In [29]:
def position(processed_entry : Doc, index : int, attachedEntity : str, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[list] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    previous_words = processed_entry[lastKeyWordIndex:index]

    LENGTH_CRITERIA = 2
    if attachedEntity == "device" :
        LENGTH_CRITERIA = 1
    position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    if not (len(position_list) == LENGTH_CRITERIA or (len(position_list) == 3 and attachedEntity == "rack")):
        position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in previous_words]))
        if not (len(position_list) == LENGTH_CRITERIA or (len(position_list) == 3 and attachedEntity == "rack")) :
            raise Exception("Wrong location format entered")
        else :
            position_list = position_list_left
    result = [float(coord) for coord in position_list] if position_list else None
    return result

In [16]:
def rotation(processed_entry : spacy.tokens.Doc, index : int, attachedEntity : str, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[float] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    previous_words = processed_entry[lastKeyWordIndex:index]

    # TODO : adapt for rack
    
    isRotationNegative = False
    rotation_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
    if not len(rotation_list) == 1 :
        rotation_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in previous_words]))
        isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
        if not len(rotation_list_left) == 1 :
            raise Exception("Wrong rotation format entered")
        else :
            rotation_list = rotation_list_left
    rotationFinal = float(rotation_list[0]) if rotation_list else None
    rotationFinal = -rotationFinal if isRotationNegative else rotationFinal
    return rotationFinal

In [17]:
def size(processed_entry : Doc, index : int, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[list] :
    # TODO : it only works for regular size not sizeXY or sizeU
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]

    size_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    if not (len(size_list) == 3):
        size_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        if not (len(size_list) == 3) :
            raise Exception("Wrong location format entered")
        else :
            size_list = size_list_left
    result = [float(coord) for coord in size_list] if size_list else None
    return result

In [12]:
def axisOrientation(processed_entry : Doc, index : int, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[list] :
    # We search the axis Orientation
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An axis Orientation  can be any combinason of [+/-]x[+/-]y. eg: +x+y or -x+y
    print(" ".join([str(token) for token in next_words]))
    axisOrientationList =  re.findall(" (-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in next_words]))
    #If we didn't find an axis orientation, we need to seek it in the second part of the sentence
    if len(axisOrientationList) ==0 :
        axisOrientationList = re.findall("(-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in last_words]))
        #If we still didn't find it, we need to raise an error
        if len(axisOrientationList) ==0 :
            raise Exception("Wrong axisOrientation format entered")
    
    #We need to raise an error if we find only one coordonate
    if len(axisOrientationList[0]) < 2:
        raise Exception("Only one coordinate was found")
    #Even if several axis Orientations where found, we always take the first one
    return axisOrientationList[0][0].replace(" ", "") + axisOrientationList[0][1].replace(" ", "")


In [13]:
def unit(processed_entry : Doc, index : int, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[list] :
    #This function help us to find the unit of the floor
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An unit can be m, t , f, meters, tildes, feets
    unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in next_words]))
    unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in next_words]))
    #If we didn't find any unit, we need to seek it in the second part of the sentence
    if len(unitList) ==0:
        unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in last_words]))
        unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in last_words]))
        
        #We should have found an unit
        if len(unitList) == 0:
            raise Exception("There wasn't any unit in the sentence")
    
    #If there are many unit, we choose the first one
    
    return unitList[0]

In [14]:
def color(processed_entry : Doc, index : int, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[list] :
    #This function seek the hexadecimal color next to the word color (or one of its synonym)
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #A color should be in hexadecimal ==> MAYBE A NAME LATER ??
    colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in next_words]))
    print(colorList)
    #If we didn't find a color after the word color, we need to look before
    if len(colorList) ==0:
        colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in last_words]))

        #If there isn't any color, we need to raise an execption
        if len(colorList) == 0: 
            raise Exception("There wasn't any color")
        
    #If there are several colors, we choose the first one
    return colorList[0].replace(" ", "")

In [15]:
testNlp = nlp("create a room called ROOM1 with an orientation of -x -y  in u with #abD874 as color   ")

print("axisOrientation : ", axisOrientation(testNlp, 0,0,22))
print("Unit : ", unit(testNlp,0,0,20))
print("Color : ", color(testNlp,20,10,25))

a room called ROOM1 with an orientation of -x -y   in u with # abD874 as color   
axisOrientation :  -x-y
Unit :  u
[]
Color :  #abD874


In [18]:
def template(processed_entry : Doc, index : int, lastKeyWordIndex : int, nextKeyWordIndex : int) -> Optional[str] :
    #The template is defined by a name, it's the same structure as for Name() but simplified since there is less convoluted wording
    
    def findClose(indexToken : int) -> Optional[int] :
        if indexToken +1 <= len(processed_entry)-1 :
            if processed_entry[indexToken+1].is_upper or processed_entry[indexToken+1].pos_ == "PROPN" :
                return indexToken+1
        if 0 <= indexToken -1 :
            if processed_entry[indexToken-1].is_upper or processed_entry[indexToken-1].pos_ == "PROPN" :
                return indexToken-1
        return None

    def findAttachedEntity(indexToken : int) -> (bool) : 
        counter = 0
        isFound = False
        currentIndex = indexToken
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = currentWord.i
            if str(currentWord) in PARAMETERS_DICT["template"] : #Peux être faut il passer aux Lemma ?
                return True
            counter += 1
        return False
    
    EXPLICIT =  PARAMETERS_DICT["name"]

    # if the name if right beside the entity
    resultIndex = findClose(index)
    if resultIndex != None :
        return processed_entry[resultIndex]
    else: #look in the NProp if one is linked to Template
        for indexTemp,tokenTemp in enumerate(processed_entry) : # look for keyword
            if processed_entry[indexTemp].is_upper or processed_entry[indexTemp].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(indexTemp)
                if findAttachedEntity(indexTemp) :
                    return tokenTemp
            
            """
            # if the token is a synonym of called
            if sum([tokenTemp.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(indexTemp)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(indexTemp)
                    if indexAttachedEntity != None :
                        resultIndex = indexAttachedEntity
                else {
                    return resultIndex
                }
            """

    return None

In [64]:
# TODO : the similarity func is very time-taking, we must shorten the process time or find another way

def main() -> str :
    FINAL_INSTRUCTION = ""

    # TODO : add already existing entity names
    ENTITIES_FULL_NAME = {"entity" : list(wiki.ENTITIES.keys())}
    KEY_WORDS_ALL = {**ENTITIES_FULL_NAME,  **PARAMETERS_DICT}

    natural_entry = input("Enter a prompt. Please follow the instructions given.\n")
    processed_entry = nlp(natural_entry)

    KEY_WORDS_ENTRY = {} 
    # we detect key words in the sentence given and put them into KEY_WORDS_ENTRY
    lastParameter = None
    for index,token in enumerate(processed_entry) :
        matching_list = [] # list of tuples with the similarity score and type of key word (for each key word)
        if token.pos_ == "VERB" and str(token) == token.lemma_ : # 2nd test : if infinitive verb
            for parameter in ACTIONS_DEFAULT.keys() :
                similarity = max([token.similarity(nlp(word)[0]) for word in ACTIONS_DEFAULT[parameter]])
                matching_list.append((similarity,parameter))
        elif token.pos_ in ["NOUN","ADP","VERB"]:
            for parameter in KEY_WORDS_ALL.keys() : 
                similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])
                matching_list.append((similarity,parameter))
        else :
            continue

        match = max(matching_list)

        # if "called" or a synonym is used for a parameter and not for an entity
        if match[1] == "name" and (lastParameter == token.head or lastParameter in token.children) :
            continue
        if match[0] > SIMILARITY_THRESHOLD :
            # if is considered a key word, is added to the dict
            KEY_WORDS_ENTRY[index] = match[1] 
            if parameter in PARAMETERS_DICT.keys() :
                lastParameter = token

    print(KEY_WORDS_ENTRY)

    dictEntities = {index : str(processed_entry[index]) for index,keyword in KEY_WORDS_ENTRY.items() if keyword == "entity"}
    print(dictEntities)

    # test detection
    list_key_param = list(KEY_WORDS_ENTRY.values())
    count_action = 0 # the nb of action words indentified
    for action_type in ACTIONS_DEFAULT.keys() :
        count_action += list_key_param.count(action_type)

    if count_action != 1 :
        raise Exception("Action not detected")
    
    # TODO : check entities
    # if no entity :check the ocli file
    global INDEX_ACTION
    global INDEX_MAIN_SUBJECT
    INDEX_ACTION = [index for index,keyword in KEY_WORDS_ENTRY.items() if keyword in ACTIONS_DEFAULT.keys()][0]
    finalRelations = findRelations(processed_entry, dictEntities, INDEX_ACTION)
    print("finalRelations : ", finalRelations)
    INDEX_MAIN_SUBJECT = findIndexMainSubject(processed_entry, KEY_WORDS_ENTRY, INDEX_ACTION, INDEX_MAIN_ENTITY)  
    print("index main subject : ", INDEX_MAIN_SUBJECT)

    INDEXES_MAIN = {"subject" : INDEX_MAIN_SUBJECT, 
                    "action" : INDEX_ACTION, 
                    "entity" : INDEX_MAIN_ENTITY}

    dictioEntityNames = name(processed_entry,
                             dictEntities,
                             [index for index,parameter in KEY_WORDS_ENTRY.items() if parameter == "name"],
                             INDEXES_MAIN)
    print("names : ",dictioEntityNames)

    if INDEX_MAIN_SUBJECT not in KEY_WORDS_ENTRY.keys() :
        # TODO
        pass
    
    # TODO : match cases (main = entity/parameter, verb = +/alteration...)
    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity" :
        # we do the processes related to each parameter
        dictioEntityParameters = wiki.makeDictParam(str(processed_entry[INDEX_MAIN_SUBJECT]))
        dictioEntityParameters["name"] = str(processed_entry[dictioEntityNames[INDEX_MAIN_SUBJECT]]).upper()
        allEntryItemsList = list(KEY_WORDS_ENTRY.items())
        for counter,(index,parameter) in enumerate(allEntryItemsList) :
            if (not parameter in PARAMETERS_DICT.keys()) or bool(dictioEntityParameters[parameter]) == True :
                continue
            lastKeyWordIndex = 0 if counter == 0 else allEntryItemsList[counter-1][0]
            nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItemsList)-1 else allEntryItemsList[counter+1][0]
            # get the parameter value
            parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
            dictioEntityParameters[parameter] = parameterValue # store the value
        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" : 
            pass
            # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_NEGATIVE" :
            pass
            # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ALTERATION" :

            

            # TODO : understand what are the changes and its consequences
            pass

    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] in PARAMETERS_DICT.keys() :
        # TODO : seek the value to be changed
        pass
    
    # if seeking the name for the main entity, pass the indexaction as parameter
    # if no name found, check the type of action : if +, a name is needed, otherwise not necessarily

    # check if parameters were not given

    # return KEY_WORDS_ENTRY

In [97]:
text = "create, in the site IMT, a building A with 0 0 position"
main()

{0: 'ALTERATION', 2: 'position', 7: 'entity', 12: 'size', 17: 'rotation', 20: 'rotation', 24: 'template'}
{7: 'building'}
finalRelations :  {}
index main subject :  2
names :  {7: 9}


In [60]:
nlp("inside")[0].similarity(nlp("having")[0])
# [nlp("current")[0].similarity(nlp(word)[0]) for word in ["axis", "orientation"]]

def hello() :
    return "hello"
type(hello)

function

In [84]:
#Test position, size, rotation, template
testNlp1 = nlp("set the position 0 0 for the building named BATIMENT with the dimension 50 40 60, turned by 90 degrees and with the template called intel640")
testNlp2 = nlp("make a room called Nowhere in the building IMT next to the building named Centrale")
testNlp3 = nlp("set the complexity of the room R1 to easy")
testNlp4 = nlp("set to easy the nbFloors of the room R1")

# print("Position :" + str(position(testNlp, 0, 0, 4, "building")))
# print("Rotation :" + str(rotation(testNlp, 15, 11, 19, "building")))
# print("Dimension :" + str(size(testNlp, 10, 6, 14, "building")))
# print("Template :" + str(template(testNlp, 22, 19, 23, "building")))

testNlp4[4].is_upper

False

In [94]:
spacy.displacy.render(testNlp1, style="dep")